In [342]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt 

In [359]:
influencer_df=pd.read_csv("influencers.csv")

In [360]:
influencer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   S.no           1050 non-null   int64 
 1   username       1050 non-null   object
 2   followers      1050 non-null   object
 3   country        1046 non-null   object
 4   viewers        1050 non-null   object
 5   Average views  1050 non-null   object
 6   Category_1     1050 non-null   object
 7   Category_2     1050 non-null   object
 8   Category_3     1050 non-null   object
dtypes: int64(1), object(8)
memory usage: 74.0+ KB


In [361]:
usersdf=pd.read_csv("SocialMediaUsers.csv")

In [362]:
influencer_df

,S.no,username,followers,country,viewers,Average views,Category_1,Category_2,Category_3
0,1,cristiano,477.9M,India,5M,6.2M,'Parenting and family','Art','History'
1,2,kyliejenner,368.1M,United States,3.5M,5.5M,'Finance and investments','Travel','Pets'
2,3,arianagrande,329.6M,United States,2.9M,4M,'Science','Movies','Beauty'
3,4,leomessi,358.6M,Indonesia,2.7M,3.5M,'Cars and automobiles','Beauty','Health and wellness'
4,5,zendaya,151.1M,United States,4.3M,5.8M,'Parenting and family','Beauty','Art'
...,...,...,...,...,...,...,...,...,...
1045,996,jadethirlwall,9.4M,United States,174.2K,228.1K,'Science','Cooking','Gardening'
1046,997,ninja,12.9M,United States,127.8K,163.9K,'Gaming','Travel','DIY and crafts'
1047,998,myriamfares,20.6M,Iraq,75.8K,102.4K,'Photography','Science','Technology'
1048,999,optimushwang,4.2M,South Korea,426K,508K,'Technology','Science','Social causes and activism'


In [358]:
influencer_df.to_csv("influencers.csv",index=False)

In [296]:
usersdf.head(10)

,UserID,Name,Gender,DOB,Interests,City,Country
0,1,Jesse Lawhorn,Female,1958-10-15,"'Movies', 'Fashion', 'Fashion', 'Books'",Sibolga,Indonesia
1,2,Stacy Payne,Female,2004-07-21,"'Gaming', 'Finance and investments', 'Outdoor ...",Al Abyār,Libya
2,3,Katrina Nicewander,Female,2000-02-07,"'DIY and crafts', 'Music', 'Science', 'Fashion'",Wādī as Sīr,Jordan
3,4,Eric Yarbrough,Male,1985-04-14,"'Outdoor activities', 'Cars and automobiles'",Matera,Italy
4,5,Daniel Adkins,Female,1955-09-18,"'Politics', 'History'",Biruaca,Venezuela
5,6,Diane Jara,Male,1967-06-18,'Travel',Belton,United States
6,7,Sheryl Morgan,Female,1969-02-09,"'Outdoor activities', 'Movies'",Haslingden,United Kingdom
7,8,William Harper,Male,1965-12-30,"'Beauty', 'Nature', 'Gardening', 'Food and din...",Ad-Damazin,Sudan
8,9,Virginia Varron,Male,1984-08-16,"'Parenting and family', 'Photography', 'Financ...",Tabuk,Saudi Arabia
9,10,Charles Figueroa,Female,2003-03-08,'Gardening',Ongole,India


In [297]:
interest_df=usersdf['Interests'].str.split(', ', expand=True)

In [298]:
interest_df.rename(columns={0:"Interest_1",1:"Interest_2",2:"Interest_3",3:"Interest_4",4:"Interest_5"},inplace=True)

In [299]:
interest_df.insert(0,"UserID",usersdf.UserID.values)

In [300]:
Users_data=pd.merge(usersdf,interest_df)

In [301]:
Users_data.drop(columns=["Interests","UserID","City"],inplace=True)

In [302]:
Users_data

,Name,Gender,DOB,Country,Interest_1,Interest_2,Interest_3,Interest_4,Interest_5
0,Jesse Lawhorn,Female,1958-10-15,Indonesia,'Movies','Fashion','Fashion','Books',None
1,Stacy Payne,Female,2004-07-21,Libya,'Gaming','Finance and investments','Outdoor activities','Travel',None
2,Katrina Nicewander,Female,2000-02-07,Jordan,'DIY and crafts','Music','Science','Fashion',None
3,Eric Yarbrough,Male,1985-04-14,Italy,'Outdoor activities','Cars and automobiles',None,None,None
4,Daniel Adkins,Female,1955-09-18,Venezuela,'Politics','History',None,None,None
...,...,...,...,...,...,...,...,...,...
99995,Lionel Denault,Female,1983-07-31,China,'DIY and crafts',None,None,None,None
99996,Margie Mieszala,Female,1954-05-20,Spain,'Cars and automobiles','Cooking','Outdoor activities',None,None
99997,Joan Mercedes,Male,1975-06-06,Chile,'Business and entrepreneurship','Cooking',None,None,None
99998,Marvin Massa,Female,1959-11-16,Brazil,'Gaming','Business and entrepreneurship','Fashion','Parenting and family','Photography'
